# Hands-on #2a: Classifying digits

In this notebook we have a look at `tf.keras`, the high-level API of TensorFlow for building and training neural networks.

Our simple task will be to classify hand-written digits:

![](img/classify_digits.svg)

We start with some imports.

In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from IPython.display import set_matplotlib_formats
%matplotlib inline
set_matplotlib_formats('svg')


## Step 1: Loading the MNIST dataset as a `tf.data.Dataset`

We use the MNIST dataset of images of hand-written digits:

In [ ]:
import tensorflow_datasets as tdfs

tdfs.disable_progress_bar()

mnist_train = tdfs.load(name='mnist', split='train')
mnist_test = tdfs.load(name='mnist', split='test')

mnist_train, mnist_test

Let us have a look at the first three images:

In [ ]:
GRAY_CMAP = plt.get_cmap('gray')

for sample in mnist_train.take(1):  # Only take a single example
    image, label = sample["image"], sample["label"]
    print(f'Label: {label}')
    plt.imshow(image[:, :, 0], cmap=GRAY_CMAP)
    plt.show()


## Step 2: Preprocessing for classification


Our first aim is to classify the digits using a neural network. We therefore

display: unable to open X server `:1' @ error/display.c/DisplayImageCommand/407.
display: unable to open X server `:1' @ error/display.c/DisplayImageCommand/407.
- scale the image tensor so that the greyscale values are between 0 and 1, and
- one-hot-encode the labels:


In [ ]:
eye = tf.eye(10, dtype='float32') # identity matrix of size 10x10


def scale_ohe(sample):
    scaled_image = tf.cast(sample['image'], 'float32') / tf.constant(255, 'float32')
    ohe_label = eye[tf.cast(sample['label'], 'int32')]
    return scaled_image,  ohe_label

Xy_train = mnist_train.map(scale_ohe)
Xy_test = mnist_test.map(scale_ohe)

print([(image.shape, label) for image, label  in Xy_train.take(4)])

To make the task more interesting, we'll add some noise to the digits:

In [ ]:
NOISE_STDDEV = 0.6
noise = tf.data.Dataset.range(1).repeat().map(lambda _: tf.random.normal((28,28,1), 0.5, NOISE_STDDEV))


In [ ]:
def add_noise(image, label):
    return image + tf.random.normal(image.shape, 0, NOISE_STDDEV), label

Xy_noisy_train = Xy_train.map(add_noise, 3) # don't leave out 3 here!
Xy_noisy_test = Xy_test.map(add_noise, 3) # don't leave out 3 here!


Let's check what the images look like now:

In [ ]:
def show_images(dataset, nr_samples=3):
    _, axes = plt.subplots(1, nr_samples)
    for i, sample in enumerate(dataset.take(nr_samples)):
        axes[i].imshow(sample[0][:,:,0], cmap=GRAY_CMAP)
    plt.show()

show_images(Xy_train)
show_images(Xy_noisy_train)

## Step 3:  Building and training a sequential model

Let us build and train a neural network to classify the digits.

First, we build it as a sequential model, that is, as a stack of layers:

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28,28,1)),
    tf.keras.layers.Dense(10, activation='softmax'),
])
model.summary()


Next, we specify how the model should learn, that is,

- which loss function should be optimized and
- which optimizer should be used.

Additionally, we declare a metrics that should be watched during training.

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy']) 

Finally, we train the model, and obtain a training history that we plot:

In [ ]:
history = model.fit(Xy_train.batch(32).take(200), validation_data=Xy_test.batch(32).take(100), epochs=5)

def plot_history(history):
    pd.DataFrame(history.history).plot.line()

plot_history(history)

Unsurprisingly, the results are quite good already.

Before we turn to the noisy data and play around with the model, we write a short function to compile and train a given model:

In [ ]:
def train(model, train_data=Xy_train, test_data=Xy_test, nr_batches=200, nr_epochs=5):
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy']) 
    history = model.fit(train_data.batch(32).take(nr_batches),
                        validation_data=test_data.batch(32).take(100),
                        epochs=nr_epochs)

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28,28,1)),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(10, activation='softmax'),
])
train(model, Xy_noisy_train, Xy_noisy_test)

## Step 4: Play around!

Now it's time for you to play around. For example, try to
- add a `'Dense'` layer between the first and the last layer with activation `'relu'` or `'sigmoid'`,
- add a `'Dropout'` layer before the last layer with a dropout rate of 0.3,
- insert 2-dimensional convolutional layers `Conv2D` before the `Flatten` layer.


In [ ]:

model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(8, kernel_size=3, input_shape=(28,28,1)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.ReLU(),
    tf.keras.layers.MaxPooling2D(pool_size=3, strides=2),
    tf.keras.layers.Conv2D(16, kernel_size=3),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.ReLU(),
    tf.keras.layers.MaxPooling2D(pool_size=3, strides=2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(10, activation='softmax'),
])
train(model, Xy_train, Xy_test, nr_batches=400)

At the end, save your model:

In [ ]:
CLASSIFIFER_PATH = 'classifier'

model.save(CLASSIFIFER_PATH, save_format='tf')